# Rede Neural de Base Radial (RBF)

As redes RBF são redes de alimentação direta (feedforward) consistindo de três camadas:


1.   **Camada de entrada**: propaga os estímulos
2.   **Camada escondida**: Unidades de processamento localmente sintonizáveis, utilizando mapeamento não linear.
3.   **Camada de saída**: Unidades de processamento lineares.


****

**O treinamento dessa rede ocorre de forma híbrida**, combinando aprendizagem não supervisionada (ANS) com a supervisionada(AS). Isso ocorre, pois em geral não se sabe quais saídas se desejam para a camada escondida. Sendo assim, a distribuição de trabalhos ocorre:
*   **ANS**: Treina a camada escondida definindo seus parâmetros livres (centros, larguras dos campos receptivos e pesos).
*   **AS**: Determina os valores dos pesos entre as camadas escondidas e de saída, considerando constantes os parâmetros já definidos.


****

**O aprendizado consiste em** determinar os valores para:
*   centro das funções de base radial,
*   largura das funções,
*   pesos da camada de saída.


Além disso, para cada neurônio da camada escondida, ele computa uma função de base radial.


Os passos necessários são:
1.   Utilizar um algoritmo ANS para encontrar os centros (protótipo para um cluster) das RBF;
2.   Utilizar métodos heurísticos para determinar a largura (área de influência de um cluster) de cada função;
3.   Utilizar um AS para determinar os pesos da camada de saída da rede.

1ª Etapa: Inicialização dos grupos com K-Means

In [1]:
!git clone https://github.com/valmirf/redes_neurais_pos.git

Cloning into 'redes_neurais_pos'...
remote: Enumerating objects: 120, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 120 (delta 11), reused 1 (delta 0), pack-reused 90 (from 1)
Receiving objects: 100% (120/120), 16.66 MiB | 20.83 MiB/s, done.
Resolving deltas: 100% (35/35), done.


Definição da função de base radial

In [2]:
# Função de base radial multiquadrática inversa
def rbfMultiquadraticaInversa(x, c, s):
    return 1 / np.sqrt((x - c)**2 + s**2)

#função de base radial gaussiana
def rbfGaussiana(x, c, s):
    return np.exp(-1 / (2 * s**2) * (x-c)**2)

#função de cálculo da largura do campo receptivo em que se repete o mesmo valor pra todos os neurônios
def computeEqualStds(centers,k):
  dist = [np.sqrt(np.sum((c1 - c2) ** 2)) for c1 in centers for c2 in centers]
  dMax = np.max(dist)
  stds = np.repeat(dMax / np.sqrt(2 * k), k)
  return stds

2ª Etapa - Treinamento de uma Rede Neural

In [5]:
from sklearn.cluster import KMeans
import numpy as np

class RBFNet(object):
    """Implementation of a Radial Basis Function Network"""

    def __init__(self, k=3, attnumber=8, lr=0.01, epochs=100, rbf=rbfGaussiana, computeStds=computeEqualStds):
        self.k = k  # grupos ou numero de neuronios na camada escondida
        self.lr = lr # taxa de aprendizagem
        self.epochs = epochs  # número de iterações
        self.rbf = rbf # função de base radial
        self.computeStds = computeStds  #função de cálculo da largura do campo receptivo

        self.w = np.random.randn(self.k,attnumber)
        self.b = np.random.randn(1)

    def fit(self, X, y):
        self.stds = []
        #K-Means pra pegar os centros inicias
        #1º parâmetro da rede RBF
        kmeans = KMeans(
            n_clusters=self.k, init='random',
            n_init=10, max_iter=300).fit(X)
        self.centers = kmeans.cluster_centers_
        #print('centers: ', self.centers)

        #Cálculo la dargura do campo receptivo
        #2º parâmetro da rede RBF
        self.stds = self.computeStds(self.centers,self.k)
        # training
        for epoch in range(self.epochs):
            for i in range(X.shape[0]):
                # forward pass
                #calcula a saída de cada neurônio da função de base radial
                phi = np.array([self.rbf(X[i], c, s) for c, s, in zip(self.centers, self.stds)])
                #calcula somatório do produto da saída da função de base radial e os pesos
                F = phi.T.dot(self.w)
                F = np.sum(F) + self.b
                #saída da rede
                out = 0 if F < 0 else 1

                #função de perda
                loss = (y[i] - out).flatten() ** 2
                #print('Loss: {0:.2f}'.format(loss[0]))

                #cálculo do erro
                error = (y[i] - out).flatten()
                #atualização dos pesos
                #3º Parâmetro da rede
                self.w = self.w + self.lr * error * phi
                self.b = self.b + self.lr * error

    #calcula saída da rede RBF com a rede treinada
    def predict(self, X):
        y_pred = []
        error = 0
        for i in range(X.shape[0]):
            a = np.array([self.rbf(X[i], c, s) for c, s, in zip(self.centers, self.stds)])
            F = a.T.dot(self.w)
            F = np.sum(F) + self.b
            out = 0 if F < 0 else 1
            y_pred.append(out)

        return np.array(y_pred)


In [10]:
# Neste código vou utilizar o pandas, framework amplamente utilizado pra lidar com dados
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

#carrega a base de dados e retorna conjuntos de treinamento e teste
def load_data():
    url = 'redes_neurais_pos/RBF/diabetes.csv'
    df = pd.read_csv(url)
    #remove a ultima coluna (dados)
    data = df[df.columns[:-1]]
    #normaliza os dados
    normalized_data = (data - data.min()) / (data.max() - data.min())
    #retorna a última coluna (rótulos)
    labels = df[df.columns[-1]]
    #separa em conjunto de treinamento e teste com seus respectivos rótulos
    X_train, X_test, y_train, y_test = train_test_split(normalized_data, labels, test_size=0.2, random_state=0)

    return X_train, X_test, y_train, y_test

#chama função que carrega base de dados
training_inputs, test_inputs, training_labels, test_labels = load_data()

#transforma rótulos do conjunto de treinamento em numeros pra calculo do erro
le = preprocessing.LabelEncoder()
le.fit(training_labels.values)
training_labels_transformed = le.transform(training_labels.values)

#chama RBF
rbfnet = RBFNet(lr=1e-2, attnumber=8, k=3, computeStds=computeEqualStds)
rbfnet.fit(training_inputs.values, training_labels_transformed)

#transforma rótulos do conjunto de teste em numeros pra calculo do erro
le = preprocessing.LabelEncoder()
le.fit(test_labels.values)
test_labels_transformed = le.transform(test_labels.values)

y_pred = rbfnet.predict(test_inputs.values) # Correção para usar dados de entrada para predição

errorabs = abs(test_labels_transformed-y_pred)

print('error: ', np.sum(errorabs)/len(test_labels_transformed))

error:  0.2662337662337662


# Descrição Mini Projeto

Utilizando o código acima, modifique a última seção (Executando com Base de Dados) para que ele seja executado com a base de dados do arquivo diabetes.csv. Depois, modifique a função de base radial implementada (Gaussiana) para a Multiquadrática Inversa e calcule a taxa de erro.


1- Calcular a quantidade de neurônios escondidos:

a) 3

b) 5

c) 7

d) 9

Qual foi a melhor configuração? Avaliaria um outro valor?

2- Utilizando a melhor configuração da questão anterior, calcular a taxa de erro usando uma das outras maneiras de retorno da largura do campo receptivo da função de base radial, em que cada neurônio possui sua própria largura.


3- Calcular a taxa de erro combinando 2 funções de Base Radial e as duas maneiras de cálculo da largura do campo receptivo:

a) Gaussiana

b) Multiquadrática Inversa


Qual foi a melhor configuração?



# 1 Questão

Criei um loop para testar ambas as Redes com funções Radiais distintas e comparar os erros.

In [16]:
# Neste código vou utilizar o pandas, framework amplamente utilizado pra lidar com dados
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

#carrega a base de dados e retorna conjuntos de treinamento e teste
def load_data():
    url = 'redes_neurais_pos/RBF/diabetes.csv'
    df = pd.read_csv(url)
    #remove a ultima coluna (dados)
    data = df[df.columns[:-1]]
    #normaliza os dados
    normalized_data = (data - data.min()) / (data.max() - data.min())
    #retorna a última coluna (rótulos)
    labels = df[df.columns[-1]]
    #separa em conjunto de treinamento e teste com seus respectivos rótulos
    X_train, X_test, y_train, y_test = train_test_split(normalized_data, labels, test_size=0.2, random_state=0)

    return X_train, X_test, y_train, y_test

# Carrega os dados
training_inputs, test_inputs, training_labels, test_labels = load_data()

# Transforma os rótulos em números
le = preprocessing.LabelEncoder()
training_labels_transformed = le.fit_transform(training_labels.values)
test_labels_transformed = le.fit_transform(test_labels.values)

# Função para testar diferentes configurações
def evaluate_rbf(k_values, rbf_function):
    errors = []
    for k in k_values:
        print(f"Treinando RBF com k={k}, função {rbf_function.__name__}...")
        rbfnet = RBFNet(lr=1e-2, attnumber=8, k=k, rbf=rbf_function, computeStds=computeEqualStds)
        rbfnet.fit(training_inputs.values, training_labels_transformed)
        y_pred = rbfnet.predict(test_inputs.values)
        errorabs = abs(test_labels_transformed - y_pred)
        error_rate = np.sum(errorabs) / len(test_labels_transformed)
        errors.append((k, error_rate))
        print(f"> Erro para k={k}: {error_rate:.4f}")
    return errors

# Testa para os valores de k com as duas funções
k_values = [3, 5, 7, 9]
print("\n--- Resultados para a Gaussiana ---")
errors_gaussiana = evaluate_rbf(k_values, rbfGaussiana)

print("\n--- Resultados para a Multiquadrática Inversa ---")
errors_multiquadratica = evaluate_rbf(k_values, rbfMultiquadraticaInversa)

# Comparação
print("\n--- Comparação de Taxas de Erro ---")
print("Gaussiana:", errors_gaussiana)
print("Multiquadrática Inversa:", errors_multiquadratica)


--- Resultados para a Gaussiana ---
Treinando RBF com k=3, função rbfGaussiana...
> Erro para k=3: 0.2468
Treinando RBF com k=5, função rbfGaussiana...
> Erro para k=5: 0.2922
Treinando RBF com k=7, função rbfGaussiana...
> Erro para k=7: 0.3377
Treinando RBF com k=9, função rbfGaussiana...
> Erro para k=9: 0.2987

--- Resultados para a Multiquadrática Inversa ---
Treinando RBF com k=3, função rbfMultiquadraticaInversa...
> Erro para k=3: 0.3247
Treinando RBF com k=5, função rbfMultiquadraticaInversa...
> Erro para k=5: 0.2143
Treinando RBF com k=7, função rbfMultiquadraticaInversa...
> Erro para k=7: 0.2338
Treinando RBF com k=9, função rbfMultiquadraticaInversa...
> Erro para k=9: 0.1948

--- Comparação de Taxas de Erro ---
Gaussiana: [(3, 0.24675324675324675), (5, 0.2922077922077922), (7, 0.33766233766233766), (9, 0.2987012987012987)]
Multiquadrática Inversa: [(3, 0.3246753246753247), (5, 0.21428571428571427), (7, 0.23376623376623376), (9, 0.19480519480519481)]


## Resultados:

--- Comparação de Taxas de Erro ---

Gaussiana: [(3, 0.24675324675324675), (5, 0.2922077922077922), (7, 0.33766233766233766), (9, 0.2987012987012987)]

Multiquadrática Inversa: [(3, 0.3246753246753247), (5, 0.21428571428571427), (7, 0.23376623376623376), (9, 0.19480519480519481)]

**Melhor Configuração:**

Para a **função Gaussiana**, a melhor configuração foi k = 3, pois apresentou o menor erro de 0.2468.

Para a **função Multiquadrática Inversa**, a melhor configuração foi k = 9, com o menor erro de 0.1948.

A configuração k = 9 para a função Multiquadrática Inversa parece ter dado o melhor desempenho entre todas as configurações. Entretanto, como k = 7 ainda teve um desempenho razoavelmente bom com erro = 0.2338, há uma chance de que valores intermediários (por exemplo, k = 8) possam produzir um bom equilíbrio entre o erro e a complexidade do modelo.

Para a RBF Gaussiana, a configuração com k = 3 é a mais eficiente, mas vale a pena testar valores próximos, como k = 4 ou k = 2, para ver se o erro pode ser reduzido ainda mais ou se existe uma diminuição significativa com a mudança de k.

# 2º Questão

Utilizando para função Multiquadrática Inversa k=9, realizei o teste com a função onde cada neurônio possui uma largura própria:

In [20]:
def computeIndividualStds(centers, *args):
    stds = []
    for i, ci in enumerate(centers):
        # Calcula a menor distância entre o centro ci e os outros centros cj
        min_dist = np.min([np.linalg.norm(ci - cj) for j, cj in enumerate(centers) if i != j])
        # Define a largura como metade dessa menor distância
        stds.append(min_dist / 2)
    return np.array(stds)

# Melhor configuração encontrada: k=9 para a Multiquadrática Inversa
rbfnet = RBFNet(lr=1e-2, attnumber=8, k=9, rbf=rbfMultiquadraticaInversa, computeStds=computeIndividualStds)

# Treinamento e teste
rbfnet.fit(training_inputs.values, training_labels_transformed)
y_pred = rbfnet.predict(test_inputs.values)

# Cálculo da taxa de erro
errorabs = abs(test_labels_transformed - y_pred)
print('Erro com largura própria para melhor configuração: ', np.sum(errorabs) / len(test_labels_transformed))


Erro com largura própria para melhor configuração:  0.3051948051948052


## Resultado:

Melhor configuração: k=9, Multiquadrática Inversa

O erro com largura própria pra cada neurônio foi: 0.3051948051948052

# 3º Questão

Calcular a taxa de erro combinando 2 funções de Base Radial e as duas maneiras de cálculo da largura do campo receptivo:

a) Gaussiana (Largura própria e Largura igual)

b) Multiquadrática Inversa (Largura própria e Largura igual)

In [22]:
# Função para treinar e avaliar a RBFNet com diferentes configurações
def evaluate_rbf_combination(rbf_function, stds_function, X_train, y_train, X_test, y_test, k_values):
    results = []
    for k in k_values:
        print(f"Treinando RBF com k={k}, função {rbf_function.__name__}, largura {stds_function.__name__}...")
        rbfnet = RBFNet(lr=1e-2, attnumber=X_train.shape[1], k=k, rbf=rbf_function, computeStds=stds_function)
        rbfnet.fit(X_train.values, y_train)
        y_pred = rbfnet.predict(X_test.values)

        # Calcula a taxa de erro
        error_rate = np.sum(abs(y_test - y_pred)) / len(y_test)
        results.append((k, error_rate))
        print(f"> Erro para k={k}: {error_rate:.4f}")
    print()
    return results

# Configurações
k_values = [3, 5, 7, 9]  # Quantidades de neurônios escondidos a serem testadas

# Avaliar combinações
print("--- Avaliação de Combinações ---")
results_gaussian_equal = evaluate_rbf_combination(rbfGaussiana, computeEqualStds, training_inputs, training_labels_transformed, test_inputs, test_labels_transformed, k_values)
results_gaussian_individual = evaluate_rbf_combination(rbfGaussiana, computeIndividualStds, training_inputs, training_labels_transformed, test_inputs, test_labels_transformed, k_values)
results_multiquad_equal = evaluate_rbf_combination(rbfMultiquadraticaInversa, computeEqualStds, training_inputs, training_labels_transformed, test_inputs, test_labels_transformed, k_values)
results_multiquad_individual = evaluate_rbf_combination(rbfMultiquadraticaInversa, computeIndividualStds, training_inputs, training_labels_transformed, test_inputs, test_labels_transformed, k_values)

# Comparar resultados
print("\n--- Resultados Finais ---")
print("Gaussiana (Largura Igual):", results_gaussian_equal)
print("Gaussiana (Largura Própria):", results_gaussian_individual)
print("Multiquadrática Inversa (Largura Igual):", results_multiquad_equal)
print("Multiquadrática Inversa (Largura Própria):", results_multiquad_individual)


--- Avaliação de Combinações ---
Treinando RBF com k=3, função rbfGaussiana, largura computeEqualStds...
> Erro para k=3: 0.2338
Treinando RBF com k=5, função rbfGaussiana, largura computeEqualStds...
> Erro para k=5: 0.2857
Treinando RBF com k=7, função rbfGaussiana, largura computeEqualStds...
> Erro para k=7: 0.3247
Treinando RBF com k=9, função rbfGaussiana, largura computeEqualStds...
> Erro para k=9: 0.3247

Treinando RBF com k=3, função rbfGaussiana, largura computeIndividualStds...
> Erro para k=3: 0.2727
Treinando RBF com k=5, função rbfGaussiana, largura computeIndividualStds...
> Erro para k=5: 0.3377
Treinando RBF com k=7, função rbfGaussiana, largura computeIndividualStds...
> Erro para k=7: 0.3312
Treinando RBF com k=9, função rbfGaussiana, largura computeIndividualStds...
> Erro para k=9: 0.3182

Treinando RBF com k=3, função rbfMultiquadraticaInversa, largura computeEqualStds...
> Erro para k=3: 0.3117
Treinando RBF com k=5, função rbfMultiquadraticaInversa, largura com

## Resultados:

--- Resultados Finais ---

**Gaussiana (Largura Igual)**: [(3, 0.23376623376623376), (5, 0.2857142857142857), (7, 0.3246753246753247), (9, 0.3246753246753247)]

**Gaussiana (Largura Própria)**: [(3, 0.2727272727272727), (5, 0.33766233766233766), (7, 0.33116883116883117), (9, 0.3181818181818182)]

**Multiquadrática Inversa (Largura Igual)**: [(3, 0.3116883116883117), (5, 0.33766233766233766), (7, 0.34415584415584416), (9, 0.2012987012987013)]

**Multiquadrática Inversa (Largura Própria)**: [(3, 0.2857142857142857), (5, 0.2792207792207792), (7, 0.3246753246753247), (9, 0.2077922077922078)]

**Melhor Configuração:**

Para a **função Gaussiana**, a melhor configuração foi **k = 3** e a largura **igual para cada neurônio**, pois apresentou o menor erro de **0.2338**.

Para a **função Multiquadrática Inversa**,  melhor configuração foi **k = 9** e a largura **igual para cada neurônio**, pois apresentou o menor erro de **0.2013**.